Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# MONAI 201 tutorial

In this tutorial we'll revisit the [MONAI 101 notebook](https://github.com/Project-MONAI/tutorials/blob/main/2d_classification/monai_101.ipynb) and add more features representing best practice concepts. This will include evaluation and tensorboard handler techniques.

These steps will be included in this tutorial, and each of them will take only a few lines of code:
- Dataset download and Data pre-processing
- Define a DenseNet-121 and run training
- Run inference using SupervisedEvaluator

This tutorial will use about 7GB of GPU memory and 10 minutes to run.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/main/2d_classification/monai_201.ipynb)

## Setup environment

In [10]:
!python -c "import monai" || pip install -q "monai-weekly[ignite, tqdm, tensorboard]"

## Setup imports

In [ ]:
import logging
import numpy as np
import os
from pathlib import Path
import sys
import tempfile
import torch
import ignite

from monai.apps import MedNISTDataset
from monai.config import print_config
from monai.data import DataLoader
from monai.engines import SupervisedTrainer, SupervisedEvaluator
from monai.handlers import (
    StatsHandler,
    TensorBoardStatsHandler,
    ValidationHandler,
    CheckpointSaver,
    CheckpointLoader,
    ClassificationSaver,
)
from monai.handlers.utils import from_engine
from monai.inferers import SimpleInferer
from monai.networks.nets import densenet121
from monai.transforms import LoadImageD, EnsureChannelFirstD, ScaleIntensityD, Compose, AsDiscreted

print_config()

## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to save results and reuse downloads.  
If not specified a temporary directory will be used.

In [12]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/workspace/Data


## Use MONAI transforms to preprocess data

We'll first prepare the data very much like in the previous tutorial with the same transforms and dataset:

In [13]:
transform = Compose(
    [
        LoadImageD(keys="image", image_only=True),
        EnsureChannelFirstD(keys="image"),
        ScaleIntensityD(keys="image"),
    ]
)

# If you use the MedNIST dataset, please acknowledge the source.
dataset = MedNISTDataset(root_dir=root_dir, transform=transform, section="training", download=True)
valdata = MedNISTDataset(root_dir=root_dir, transform=transform, section="validation", download=False)

2024-02-27 08:31:31,955 - INFO - Verified 'MedNIST.tar.gz', md5: 0bc7306e7427e00ad1c5526a6677552d.
2024-02-27 08:31:31,955 - INFO - File exists: /workspace/Data/MedNIST.tar.gz, skipped downloading.
2024-02-27 08:31:31,956 - INFO - Non-empty folder exists in /workspace/Data/MedNIST, skipped extracting.


Loading dataset:   0%|          | 0/47164 [00:00<?, ?it/s]

Loading dataset: 100%|██████████| 5895/5895 [00:02<00:00, 2465.05it/s]


## Define a network and a supervised trainer

For training we have the same elements again and will slightly change the `SupervisedTrainer` by expanding its train_handlers. This upgrade will be beneficial for efficient utilization of TensorBoard.
Furthermore, we introduce a `SupervisedEvaluator` object that will efficiently track model progress. Accompanied by `TensorBoardStatsHandler`, it will log statistics for TensorBoard, ensuring precise tracking and management.

In [14]:
max_epochs = 5
save_interval = 2
out_dir = "./eval"
model = densenet121(spatial_dims=2, in_channels=1, out_channels=6).to("cuda:0")

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

evaluator = SupervisedEvaluator(
    device=torch.device("cuda:0"),
    val_data_loader=DataLoader(valdata, batch_size=512, shuffle=False, num_workers=4),
    network=model,
    inferer=SimpleInferer(),
    key_val_metric={"val_acc": ignite.metrics.Accuracy(from_engine(["pred", "label"]))},
    val_handlers=[StatsHandler(iteration_log=False), TensorBoardStatsHandler(iteration_log=False)],
)

trainer = SupervisedTrainer(
    device=torch.device("cuda:0"),
    max_epochs=max_epochs,
    train_data_loader=DataLoader(dataset, batch_size=512, shuffle=True, num_workers=4),
    network=model,
    optimizer=torch.optim.Adam(model.parameters(), lr=1e-5),
    loss_function=torch.nn.CrossEntropyLoss(),
    inferer=SimpleInferer(),
    train_handlers=[
        ValidationHandler(validator=evaluator, epoch_level=True, interval=1),
        CheckpointSaver(
            save_dir=out_dir,
            save_dict={"model": model},
            save_interval=save_interval,
            save_final=True,
            final_filename="checkpoint.pt",
        ),
        StatsHandler(),
        TensorBoardStatsHandler(tag_name="train_loss", output_transform=from_engine(["loss"], first=True)),
    ],
)

## Run the training

In [ ]:
trainer.run()

## View training in tensorboard

Please uncomment the following cell to load tensorboard results.

In [2]:
# %load_ext tensorboard
# %tensorboard --logdir ./runs

## Inference

First thing to do is to prepare the test dataset:

In [6]:
dataset_dir = Path(root_dir, "MedNIST")
class_names = sorted(f"{x.name}" for x in dataset_dir.iterdir() if x.is_dir())
testdata = MedNISTDataset(root_dir=root_dir, transform=transform, section="test", download=False, runtime_cache=True)

Next, we're going to establish a `SupervisedEvaluator`. This evaluator will process all the files in the specified directory and persist the results into a CSV file. Validation handlers (val_handlers) will be utilized to load the checkpoint file, providing an error if any file is unavailable, and they will also save the classification outcomes.

In [10]:
evaluator = SupervisedEvaluator(
    device=torch.device("cuda:0"),
    val_data_loader=DataLoader(testdata, batch_size=1, num_workers=0),
    network=model,
    inferer=SimpleInferer(),
    postprocessing=AsDiscreted(keys="pred", argmax=True),
    val_handlers=[
        CheckpointLoader(load_path=f"{out_dir}/checkpoint.pt", load_dict={"model": model}),
        ClassificationSaver(
            batch_transform=lambda batch: batch[0]["image"].meta, output_transform=from_engine(["pred"])
        ),
    ],
)

evaluator.run()

INFO:ignite.engine.engine.SupervisedEvaluator:Engine run resuming from iteration 0, epoch 0 until 1 epochs
INFO:ignite.engine.engine.SupervisedEvaluator:Restored all variables from ./eval/checkpoint.pt
INFO:ignite.engine.engine.SupervisedEvaluator:Epoch[1] Complete. Time taken: 00:01:24.338
INFO:ignite.engine.engine.SupervisedEvaluator:Engine run complete. Time taken: 00:01:24.390


By default, the inference results are stored in a file named "predictions.csv". However, this output filename can be customized within the `ClassificationSaver` handler, according to your preferences.
Upon examining the output, one can note that the second column corresponds to the predicted class. A more discernable interpretation can be achieved by using these values as indices mapped to our predefined list of class names.

In [12]:
max_items_to_print = 10
for fn, idx in np.loadtxt("./predictions.csv", delimiter=",", dtype=str):
    print(fn, class_names[int(float(idx))])
    max_items_to_print -= 1
    if max_items_to_print == 0:
        break

/workspace/Data/MedNIST/AbdomenCT/006070.jpeg AbdomenCT
/workspace/Data/MedNIST/BreastMRI/006574.jpeg BreastMRI
/workspace/Data/MedNIST/ChestCT/009858.jpeg ChestCT
/workspace/Data/MedNIST/CXR/007398.jpeg CXR
/workspace/Data/MedNIST/Hand/005663.jpeg Hand
/workspace/Data/MedNIST/HeadCT/006896.jpeg HeadCT
/workspace/Data/MedNIST/HeadCT/007179.jpeg HeadCT
/workspace/Data/MedNIST/CXR/001190.jpeg CXR
/workspace/Data/MedNIST/ChestCT/005138.jpeg ChestCT
/workspace/Data/MedNIST/BreastMRI/000023.jpeg BreastMRI
